In [1]:
import numpy as np
import random
from time import sleep
thet_l = []
mistake = []
# m denotes the number of examples here, not the number of features
def gradientDescent(x, y, theta, alpha, m, numIterations):
    global thet_l, mistake
    xTrans = x.transpose()
    for i in range(0, numIterations):
        hypothesis = np.dot(x, theta)
        loss = hypothesis - y
        # avg cost per example (the 2 in 2*m doesn't really matter here.
        # But to be consistent with the gradient, I include it)
        cost = np.sum(loss ** 2) / (2 * m)
        mistake.append(cost)
        #print("Iteration %d | Cost: %f" % (i, cost))
        # avg gradient per example
        gradient = np.dot(xTrans, loss) / m
        # update
        theta = theta - alpha * gradient
        thet_l.append(theta)
    return theta


def genData(numPoints, bias, variance):
    X = np.zeros(shape=numPoints)
    Y = np.zeros(shape=numPoints)
    for i in range(0, numPoints):
        # bias feature
        X[i] = i
        # our target variable
        Y[i] = (i + bias) + random.uniform(0, 1) * variance
    return X, Y




In [2]:
X, Y = genData(100, 25, 10)


In [3]:
import random
import math
 
coefs = []
losses = []
    
def local_grad(x, y, a, b):
    return 2 * x ** 2 * a + 2 * x * b - 2 * x * y, 2 * b + 2 * x * a - 2 * y
 
def grad(X, Y, a, b):
    gr_a = 0
    gr_b = 0
    for i in range(len(X)):
        gr = local_grad(X[i], Y[i], a, b)
        gr_a += gr[0]
        gr_b += gr[1]
    mdl = (gr_a ** 2 + gr_b ** 2) ** (1 / 2)
    gr_a /= mdl
    gr_b /= mdl
    return gr_a, gr_b
 
def loss(X, Y, a, b):
    loss = 0
    for i in range(len(X)):
        loss += (a * X[i] + b - Y[i]) ** 2
    return loss
 
def local_min_step(x, y, a, b, gr_a, gr_b):
    k = gr_a * x + gr_b
    m =  a * x + b - y
    lambda_coef = k ** 2
    free_coef = k * m
    return lambda_coef, free_coef
 
def min_step(X, Y, a, b, gr_a, gr_b):
    lambda_coef = 0
    free_coef = 0
    for i in range(len(X)):
        coef = local_min_step(X[i], Y[i], a, b, gr_a, gr_b)
        lambda_coef += coef[0]
        free_coef += coef[1]
    return -free_coef / lambda_coef
 
def GDS(X, Y, iter):
    global coefs, losses
    coefs = []
    losses = []
    a = 1
    b = 0
    grad_a, grad_b = 1, 1
    lambda_step = 1
    a_best, b_best = a, b
    best = loss(X, Y, a, b)
    num = [i for i in range(len(X))]
    random.shuffle(num)
    #print(num)
    i = 0
    C = len(X)
    coefs.append((a, b))
    losses.append(best)
    for k in range(iter):
        #print(loss(X, Y, a, b))
        if (best > loss(X, Y, a, b)):
            best = loss(X, Y, a, b)
            a_best, b_best = a, b
        #print(a, b)
        grad_a, grad_b = local_grad(X[i % len(X)], Y[i % len(X)], a, b)
        mdl = (grad_a ** 2 + grad_b ** 2) ** (1 / 2)
        if (mdl == 0):
            continue        
        grad_a /= mdl
        grad_b /= mdl
        #print(grad_a, grad_b)
        coef = local_min_step(X[i % len(X)], Y[i % len(X)], a, b, -grad_a, -grad_b)
        if (coef[0] == 0):
            continue
        lambda_step = -coef[1] / coef[0]
        C = 1 / (i + 1) ** (3 / 5)
        a -= lambda_step * C * grad_a
        b -= lambda_step * C * grad_b
        coefs.append((a, b))
        losses.append(loss(X, Y, a, b))
        i += 1
    #print(a_best, b_best)
    #print(best)
    return a_best, b_best
 
def GD(X, Y, iter):
    global coefs, losses
    coefs = []
    losses = []
    a = 1
    b = 0
    grad_a, grad_b = 1, 1
    lambda_step = 1
    coefs.append((a, b))
    losses.append(loss(X, Y, a, b))
    for k in range(iter):
        #print(loss(X, Y, a, b))
        #print(a, b)
        grad_a, grad_b = grad(X, Y, a, b)
        #print(grad_a, grad_b)
        lambda_step = min_step(X, Y, a, b, -grad_a, -grad_b)
        a -= lambda_step * grad_a
        b -= lambda_step * grad_b
        coefs.append((a, b))
        losses.append(loss(X, Y, a, b))
    #print(a, b)
    return a, b

In [4]:
iter = 10000
GD(X, Y, iter)

(0.99385811093545451, 29.982462197358526)

In [15]:
iter = 100
GDS(X, Y, iter)

[73, 62, 7, 6, 36, 28, 68, 20, 55, 61, 54, 93, 77, 70, 19, 88, 52, 48, 60, 41, 49, 30, 51, 42, 23, 85, 27, 94, 3, 10, 89, 57, 79, 22, 2, 38, 90, 35, 8, 75, 11, 99, 37, 95, 84, 78, 50, 40, 17, 43, 5, 39, 53, 14, 67, 58, 81, 63, 56, 34, 26, 98, 18, 72, 4, 15, 0, 92, 82, 9, 47, 86, 97, 31, 21, 71, 29, 66, 64, 25, 74, 1, 91, 87, 45, 46, 13, 80, 69, 76, 33, 96, 16, 83, 59, 44, 32, 65, 24, 12]


(1.0, 30.665966141759444)

In [6]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
fig = plt.figure()
ax1 = fig.add_subplot(2,1,2)
ax2 = fig.add_subplot(2,1,1)
ax1.plot(X,Y, 'ro')
def animate(i):
    ax1.clear()
    ax2.plot(i, losses[i], 'go')
    ax1.text(80, 20,str(i))
    ax1.plot(X, Y, 'ro')
    ax1.plot(X, coefs[i][0]*X+coefs[i][1], 'g')
    plt.draw()
ani = animation.FuncAnimation(fig, animate, interval=10000)
plt.show()